# Neural Style Transfer

## Overview

In this assignment, you will complete the missing pieces of a TensorFlow implementation of the Neural Style Transfer algorithm, which allows you to apply the style of one image to the content of another.  The algorithm is based on a pretrained convolutional neural network model called VGG-19 [\[4\]](#refs), which we will be using to extract low-level and high-level visual features from images.

By exploiting the fact that low-level visual features correspond roughly to what we think of as "style," and high-level features correspond roughly to "content," we use these two kinds of features to define a measure of style similarity and content similarity between any two pairs of images.  Then, once we've defined these similarity measures, we can use them to generate images which with style similar to one input image, but content similar to another.  We generate these images based on the iterative optimization process of gradient descent, gradually refining the pixels of a candidate image to achieve the desired style and content.


# Credit

This assignment is adapted from [Raymond Yuan's excellent tutorial on Neural Style Transfer using TensorFlow](https://medium.com/tensorflow/neural-style-transfer-creating-art-with-deep-learning-using-tf-keras-and-eager-execution-7d541ac31398) [\[5\]](#refs).  Most of the code in this assignment, and some of the explanations, originally appeared in that tutorial.

The original tutorial contains no exercises or missing code sections.  Our contributions consist of converting this tutorial to a multi-part course assignment with questions, coding problems, and hints, as well as adding Questions 4 and 5 to allow the student to interactively explore the effect of different hyperparameter choices on the model.  The ideas which are explored in Questions 4 and 5 are based on ideas described in the original paper on Neural Style Transfer by Gatys et al [\[3\]](#refs). The implementation using TensorFlow Hub model in Question 6 is taken and modified from the tensorflow tutorial [\[6\]](#refs).

## Learning Objectives
After this project, you should feel comfortable with the following:

* Running pretrained convolutional neural network (CNN) models
* Understanding how different layers of a CNN process different features of an image
* Using pretrained models to extract features for use in other algorithms
* Using latent features to define similarity measurements between images
* Understanding the design of the Neural Style Transfer algorithm, including:
  * the loss functions it defines
  * how it uses gradient descent
  * the effect of different choices of hyperparameters


## Setup

### Import and configure modules

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models 
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

### Download Images

In [ ]:
import os
img_dir = '/tmp/nst'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)

!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/d/d7/Green_Sea_Turtle_grazing_seagrass.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg

!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg
!wget --quiet -P /tmp/nst/ https://courses.byui.edu/art110_new/art110/week06/images/sphere_large.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Rembrandt_Harmenszoon_van_Rijn_-_An_Old_Man_in_Red.JPG/1920px-Rembrandt_Harmenszoon_van_Rijn_-_An_Old_Man_in_Red.JPG
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Oski_pregame_at_UCLA_at_Cal_2008-10-25.JPG/460px-Oski_pregame_at_UCLA_at_Cal_2008-10-25.JPG
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg

In [ ]:
# enable eager execution such that "operations return concrete values instead of constructing a computational graph to run later."
# https://www.tensorflow.org/guide/eager
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
# Set up some global values here
content_path = '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg'
style_path = '/tmp/nst/The_Great_Wave_off_Kanagawa.jpg'

## Visualize the input

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

In [ ]:
def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

These are input content and style images. We hope to "create" an image with the content of our content image, but with the style of the style image. 

In [ ]:
plt.figure(figsize=(10,10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style, 'Style Image')
plt.show()

## Prepare the data
Let's create methods that will allow us to load and preprocess our images easily. We perform the same preprocessing process as are expected according to the VGG training process. VGG networks are trained on image with each channel normalized by `mean = [103.939, 116.779, 123.68]`and with channels BGR.

In [ ]:
def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

In order to view the outputs of our optimization, we are required to perform the inverse preprocessing step. Furthermore, since our optimized image may take its values anywhere between $- \infty$ and $\infty$, we must clip to maintain our values from within the 0-255 range.   

In [ ]:
def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

## Q1 Build the Model



We want to build a model which allows us to get some feature representation values so that we can use them to measure how similar two images are in terms of their content and style.  We will use the hidden layers of the [VGG19](https://keras.io/applications/#vgg19) model as a source of both content and style features.

We will be using the VGG19 model to build a wrapper model which returns the feature maps generated in its hidden layers.  We can construct a TensorFlow model with a given set of input and output layers with the call:
`model = Model(inputs, outputs)`

**a. Your job is to print all the layers in vgg model.**

Hint: You can access layers using YOUR_MODEL.layers.

In [ ]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

# print all the layers in VGG19
# BEGIN YOUR SOLUTION 

# END YOUR SOLUTION

### Choose intermediate layers 
Now we choose intermediate layers from the network to represent the style and content of the image. Later (in Q5) you will explore the effects of using differnt subsets of these layers.

In [ ]:
# no need to change this cell

content_layers = ['block5_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

**b. Your job is to complete the get_vgg_model().** We need to specify the `model_outputs`, a list of output values. We also need to set the model's attribute 'trainable' to false because we are not training the vgg model.

Hints: 
1. Load a pretrained VGG model. We just did this in the previous cell. The model has an boolean attribute named 'trainable'. You can modify this attribute directly.
2. Use `MODEL.get_layer(LAYER_NAME).output` to get a output layer named `LAYER_NAME` in `MODEL`. Which model are we using? What layers do we want?

In [ ]:
def get_vgg_model(style_layers, content_layers):
  """ Creates a vgg model that returns a list of intermediate output values. """
  
  # Load pretrained VGG model, trained on imagenet data
  # Set the trainable attribute of your model to False
  # BEGIN YOUR SOLUTION

  # END YOUR SOLUTION

  

  # Get output layers corresponding to style and content layers
  # BEGIN YOUR SOLUTION 

  # END YOUR SOLUTION

  # Build model  
  return models.Model(vgg.input, model_outputs)

##Q2 Define and create our loss functions (content and style distances)

To generate an image that matches the content of the content image and the style of the style image, We need to define some loss functions for content and style of our images so that we can use optimizer to minimize these losses 

### Content Loss

We will define the content loss as the squared Euclidean distance between two content feature maps, viewing each feature of each pixel as a distinct coordinate.  Let $F^l_{ij}(x)$ and $P^l_{ij}(p)$ denote the respective internal feature maps produced by layer $l$ of the network when evaluated with a candidate image $x$ and content source image $p$. The content loss is then defined as: $$L^l_{content}(p, x) = \sum_{i, j} (F^l_{ij}(x) - P^l_{ij}(p))^2$$ 

**a. Your job is to complete `get_content_loss()`.**

Hint: Functions you may need: `tf.reduce_mean` and `tf.square`. 

In [ ]:
def get_content_loss(base_content_feature, target_feature):
  """ Compute the content loss of two feature values.
    Arguments:
      base_content_feature: feature value of the base content image in some layer.
      target_feature: feature value of the target image content in some layer.

    Returns the content loss between two input feature maps.
  """
  # BEGIN YOUR SOLUTION 

  # END YOUR SOLUTION

## Style Loss

To compute the difference in style between two images, we don't use the direct squared Euclidean distance between feature maps.  Instead, we compute a kind of summary of each image's distribution of style features, and then compare these summaries.  Specifically, we will summarize each image's feature map at a given layer using its *Gram matrix*, which represents the correlation between different features across all pixels.  The Gram matrix for the candidate image is defined as:
$$ G^l_{ij} = \sum_k F^l_{ik} F^l_{jk} $$
where the indices $i, j$ range over feature layers, and the index $k$ ranges over pixels.  The Gram matrix $A$ for the style source image is defined analogously.  The style loss is then defined as
$$ E_l = \frac{1}{4 N_l^2 M_l^2} \sum_{i,j} (G^l_{ij} - A^l_{ij})^2 $$
(Hint: the normalizing factor $N_l^2 M_l^2$ there is just to make the equation into a mean instead of a sum.  You won't need to include it explicitly in your code if you use the right function)



### Computing style loss


**b. Your job is to complete `get_style_loss()`.** You have done something similar in last question.

In [ ]:
def gram_matrix(input_tensor):
  """ Returns the gram matrix of the input tensor."""
  # We make the image channels first 
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  # We scale the loss at a given layer by the size of the feature map and the number of filters
  height, width, channels = base_style.get_shape().as_list()
  
  gram_style = gram_matrix(base_style)
  
  # BEGIN YOUR SOLUTION

  # END YOUR SOLUTION

## Q3 Apply style transfer to our images


### Run Gradient Descent 

Neural Style Transfer works by iteratively adjusting a *candidate output image* using gradient descent to minimize the style and content losses.  Note that this is different from many common applications of gradient descent in ML: we are updating an image, *not* the weights of any neural network!



We have defined a helper function `get_feature_representations` to compute our content and style feature representations for you. 

In [ ]:
def get_feature_representations(model, content_path, style_path, num_style_layers):
  """Helper function to compute our content and style feature representations.

  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: an image classifier model
    content_path: The path to the content image.
    style_path: The path to the style image
    num_style_layers: Number of style layers
    
  Returns:
    returns the style features and the content features. 
  """
  # Load images
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)
  
  # batch compute content and style features
  style_outputs = model(style_image)
  content_outputs = model(content_image)
  
  
  # Get the style and content feature representations from our model  
  style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
  content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
  return style_features, content_features

### Computing the loss and gradients


We compute the total loss by accumulating content loss and style loss from each layer. We equally weight each contribution of each loss layer. However, before adding both losses to the final total loss, we weight the total content loss and total style loss differently. That's why we add the argument `loss_weights`, a pair of weights for content loss and style loss to the `compute_loss` function. We will explore more about this parameter in the "Content/Style Ratio Effects" section.

**a. Your job is to complete `compute_loss()`.**

Hint: We have provided an example of how to do it for style loss before the part you need to implement. You need to do the similar thing for content loss. 

In [ ]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features, num_content_layers, num_style_layers):
  """Computes the total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight = loss_weights
  
  # Feed our init image through our model. This will give us the content and 
  # style representations at our desired layers. Since we're using eager execution
  # our model is callable just like any other function!
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
  
  # Accumulate content losses from all layers 
  # BEGIN YOUR SOLUTION

  # END YOUR SOLUTION

  # weight each type of loss and get the total loss
  # BEGIN YOUR SOLUTION

  # END YOUR SOLUTION 
  return total_loss, style_score, content_score

Then we want to compute the gradient. We apply the gradients of the loss we are calculating to optimize the initial base image.

We use TensorFlow's [**tf.GradientTape**](https://www.tensorflow.org/programmers_guide/eager#computing_gradients) feature to compute the gradient.  You don't need to understand much about how this works, but you should know that it puts TensorFlow in a mode where it automatically tracks the gradients of tensors that you compute with respect to the parameters you're trying to optimize.

**b. Your job is to complete `compute_grads_and_total_loss()`**

Hint: Below is an example taken from the documentation of [**tf.GradientTape**](https://www.tensorflow.org/programmers_guide/eager#computing_gradients). Try to do some pattern matching. Remember we are updating the base image(`cfg['init_image']`) instead of some other weights. "**" can unpack arguments stored in a dictionary.

```
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
  loss = w * w

grad = tape.gradient(loss, w)
```

In [ ]:
def compute_grads_and_total_loss(cfg):
  """Computes the gradients.
  Arguments: 
  cfg: a dictionary of configuration variables passed in compute_loss()
  Returns: gradent object, 
  """
  
  # BEGIN YOUR SOLUTION

  # END YOUR SOLUTION
  return grad, all_loss

### Optimization loop

Now we put everything together to define our `run_style_transfer()` function. The function takes the paths to a content image and a style image, lists of layer names we use for feature extraction(we specified the layers in the cell right before the 'Build the Model' section) and the relative weights for content and style losses and output the best image and best loss after doing some optimization over the loss.
We will use Adaptive Moment Estimation(Adam), a gradient descent optimization algorithm, optimizer to minimize the loss. You are not required to undertand how Adam works, but you should be able to follow the instructions to figure out how and where to use it.

**c. Your job is to complete run_style_transfer().**

Hint: Here are a few things you need to do:

1.   You need to call a function we defiend in previous part to compute the gradients and total loss.
2.   You need to create an optimizer using `tf.train.AdamOptimizer` with `learning_rate=5`, `beta1=0.99` and `epsilon=1e-1`.
3.   You need to use this optimizer to apply the processed gradients with [**`apply_gradients()`**](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Optimizer#apply_gradients). Note that `apply_gradients()` takes a list of tuples. In our case, we only need to provide one tuple: (grads, variable_we_try_to_update). Which variable are we trying to apply the gradients to, or what do we need to update?
4.   You need to add an if statement to compare the losses before updating the `best_loss` and `best_image`. We are trying to minimize the loss to get the best image. To update the best_image, you can just use `deprocess_img(init_image.numpy())`.






In [ ]:
import IPython.display

def run_style_transfer(content_path, 
                       style_path,
                       content_layers,
                       style_layers,
                       num_iterations=1000,
                       content_weight=1e3, 
                       style_weight=1e-2,
                       show_images=False): 
  # We don't need to (or want to) train any layers of our model, so we set their
  # trainable to false. 
  model = get_vgg_model(content_layers, style_layers) 
  for layer in model.layers:
    layer.trainable = False
  
  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_feature_representations(model, content_path, style_path, len(style_layers))
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  
  # Set initial image
  init_image = load_and_process_img(content_path)
  init_image = tf.Variable(init_image, dtype=tf.float32)

  # Create our optimizer
  # BEGIN YOUR SOLUTION

  # END YOUR SOLUTION

  # For displaying intermediate images 
  iter_count = 1
  
  # Store our best result
  best_loss, best_img = float('inf'), None
  
  # Create a nice config 
  loss_weights = (style_weight, content_weight)
  cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features,
      'num_content_layers': len(content_layers),
      'num_style_layers': len(style_layers),
  }
    
  # For displaying
  num_rows = 2
  num_cols = 5
  display_interval = num_iterations/(num_rows*num_cols)
  start_time = time.time()
  global_start = time.time()
  
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   
  
  imgs = []

  # iteratively update the base image based on the loss
  for i in range(num_iterations):

    # compute the gradients and total loss
    # BEGIN YOUR SOLUTION
    # END YOUR SOLUTION

    total_loss, style_score, content_score = all_losses

    # Apply the gradient
    # BEGIN YOUR SOLUTION
    # END YOUR SOLUTION

    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time() 
    
    # Update best loss and best image from total loss.
    # BEGIN YOUR SOLUTION
    # END YOUR SOLUTION


    if i % display_interval== 0:
      start_time = time.time()
      
      # Use the .numpy() method to get the concrete numpy array
      plot_img = init_image.numpy()
      plot_img = deprocess_img(plot_img)
      imgs.append(plot_img)
      IPython.display.clear_output(wait=True)
      if show_images:
        IPython.display.display_png(Image.fromarray(plot_img))
      print('Iteration: {}'.format(i))        
      print('Total loss: {:.4e}, ' 
            'style loss: {:.4e}, '
            'content loss: {:.4e}, '
            'time: {:.4f}s'.format(total_loss, style_score, content_score, time.time() - start_time))
  print('Completed! Total time: {:.4f}s'.format(time.time() - global_start))

  # Display all intermediate output images at the specified iteration interval 
  if show_images:
    IPython.display.clear_output(wait=True)
    plt.figure(figsize=(14,4))
    for i,img in enumerate(imgs):
        plt.subplot(num_rows,num_cols,i+1)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
      
  return best_img, best_loss 

In [ ]:
# It may take 2-3 minutes to run this function
best_img, best_loss = run_style_transfer(content_path, 
                                     style_path, content_layers, style_layers, show_images=True)

In [ ]:
Image.fromarray(best_img)

## Visualize outputs
We "deprocess" the output image in order to remove the processing that was applied to it. 

In [ ]:
def show_results(best_img, content_path, style_path, show_large_final=True):
  plt.figure(figsize=(10, 5))
  content = load_img(content_path) 
  style = load_img(style_path)

  plt.subplot(1, 2, 1)
  imshow(content, 'Content Image')

  plt.subplot(1, 2, 2)
  imshow(style, 'Style Image')

  if show_large_final: 
    plt.figure(figsize=(10, 10))

    plt.imshow(best_img)
    plt.title('Output Image')
    plt.show()

In [ ]:
show_results(best_img, content_path, style_path)

## Q4 Content/Style Ratio Effects

**Your job is to run the following cell and answer the following question.**

Question: Any noticible difference you found from the following output images? What causes this difference?

**BEGIN YOUR SOLUTION**

 

**END YOUR SOLUTION**

In [ ]:
# Depending on how many pairs of weights you try, running this cell may take "number of pairs" * 3 mins/pair
content_style_weights = [(1e3, 1e4),(1e3, 1e2),(1e3, 1e-2),(1e5, 1e-2),(1e7, 1e-2)]
plt.figure(figsize=(20, 10))

for i, weights in enumerate(content_style_weights):
  c_w = weights[0]
  s_w = weights[1]
  print('content/style: ' + str(c_w/s_w))

  best_img, best_loss = run_style_transfer(content_path, 
                       style_path,
                       content_layers,
                       style_layers,
                       content_weight=c_w, 
                       style_weight=s_w)
  


  plt.subplot(len(content_style_weights), 1, i+1)
  plt.imshow(best_img)
  plt.title('content/style: ' + str(c_w/s_w))
plt.tight_layout()
plt.show()


## Q5 Effects of Using Different Subsets of CNN Layers

Now we want to exmaine the effects of using different subsets of CNN layers for style.

**a. Your job is to try some combinations of layers in the predefined variable `style_layer`in the section "Choose intermediate layers" and run the code in the following cell. Then describe what you observed.**

Hint: Doing this with increasing subsets in order may help you understand the effects easier.

**BEGIN YOUR SOLUTION**



**END YOUR SOLUTION**

In [ ]:
# don't change this variable
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

best_imgs = [] # best images for different subsets of style layers used in style transfer
style_layers_list = [] # list of lists storing subsets of style layers

# Save best images in best_imgs
# BEGIN YOUR SOLUTION

# END YOUR SOLUTION

# display all best images
for best_img in best_imgs:
  plt.figure(figsize=(8, 8))
  plt.imshow(best_img)
  plt.title('Output Image with Style Layers: ' + str(style_layers_list))
  plt.show()

## Q6 Style Transfer using TensorFlow Hub model

We have worked through the implementation of neural style transfer. Now we would like to see how we can make use of TensorFlow Hub, a repository of trained machine learning models, to perform the whole style transfer for us.

In [ ]:
def tf_load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def tf_imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return Image.fromarray(tensor)
  
# content_image = tf_load_img(content_path)
# style_image = tf_load_img(style_path)

# plt.subplot(1, 2, 1)
# tf_imshow(content_image, 'Content Image')

# plt.subplot(1, 2, 2)
# tf_imshow(style_image, 'Style Image')

**a. Your job is to run the following cell and answer th following questions.**

Questions: Did you get the same styled image as we got in Q3? If you see any difference, what may cause this difference? 

Hint: Think about what we have explored in Q4 and Q5 and the parts we have done in the implementation.

**BEGIN YOUR SOLUTION**


**END YOUR SOLUTION**

In [ ]:
import tensorflow_hub as hub

content_image = tf_load_img(content_path)
style_image = tf_load_img(style_path)
 
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]


tensor_to_image(stylized_image)

## Q7 Try it on other images

Now you know how to do the neural style transfer! 

**a. Your job is to try this technique on other images and download your favorite one to submit**. 

To download an image from Colab, you may find the following code in comments useful.

In [ ]:
#from google.colab import files
#files.download('artwork.png')

# BEGIN YOUR SOLUTION

# END YOUR SOLUTION

### Starry night + Sphere

In [ ]:
best_starry_sphere, best_loss = run_style_transfer('/tmp/nst/sphere_large.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg', content_layers, style_layers, show_images=False)

In [ ]:
show_results(best_starry_sphere, '/tmp/nst/sphere_large.jpg',
                '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

### Pillars of Creation + Sphere

In [ ]:
best_pillars_sphere, best_loss = run_style_transfer('/tmp/nst/sphere_large.jpg',
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg', content_layers, style_layers, show_images=False)

In [ ]:
show_results(best_pillars_sphere, '/tmp/nst/sphere_large.jpg',
                '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

### Rembrandt + Oski

In [ ]:
best_rembrandt_oski, best_loss = run_style_transfer('/tmp/nst/460px-Oski_pregame_at_UCLA_at_Cal_2008-10-25.JPG', 
                                                  '/tmp/nst/1920px-Rembrandt_Harmenszoon_van_Rijn_-_An_Old_Man_in_Red.JPG',
                                                  content_layers, style_layers, show_images=False)


In [ ]:
show_results(best_rembrandt_oski, 
             '/tmp/nst/460px-Oski_pregame_at_UCLA_at_Cal_2008-10-25.JPG', 
             '/tmp/nst/1920px-Rembrandt_Harmenszoon_van_Rijn_-_An_Old_Man_in_Red.JPG', )

<a name='refs'></a>

# References

\[1\] **[Image of Green Sea Turtle](https://commons.wikimedia.org/wiki/File:Green_Sea_Turtle_grazing_seagrass.jpg)**
By P.Lindgren [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], from Wikimedia Commons.

\[2\] **[Image of Oski](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Oski_pregame_at_UCLA_at_Cal_2008-10-25.JPG/460px-Oski_pregame_at_UCLA_at_Cal_2008-10-25.JPG)**
By BrokenSphere [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], from Wikimedia Commons.

\[3\] **[Image of Ball and Shadow](https://courses.byui.edu/art110_new/art110/week06/images/sphere.jpg)**. Brigham Young University—Idaho, Department of Art, Art 110, 2010. \url{https://courses.byui.edu/art110_new/art110/week06/images/sphere.jpg}

\[4\] Gatys, L. A., Ecker, A. S., Bethge, M. et al. "A neural algorithm of artistic style." arXiv preprint, 2015, https://arxiv.org/abs/1508.06576.

\[5\] Simonyan, K., Zisserman, A. "Very Deep Convolutional Networks for Large-Scale Image Recognition." arXiv preprint, 2015, https://arxiv.org/pdf/1409.1556.

\[6\] Yuan, R. "Neural Style Transfer: Creating Art with Deep Learning using tf.keras and eager execution." Medium, 3, Aug. 2018, https://medium.com/tensorflow/neural-style-transfer-creating-art-with-deep-learning-using-tf-keras-and-eager-execution-7d541ac31398.

\[7\] "Neural style transfer." TensorFlow.org, 2020, https://www.tensorflow.org/tutorials/generative/style_transfer.

\[8\] All other images from Wikimedia Commons. 